In [3]:
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import cv2

# Torch常用運算

In [58]:
name = ['B', 'C', 'H', 'W']
x = torch.rand((2, 3, 3, 3), names=name)
x_sum = x.sum(dim='C', keepdim=True)
x_0 = x.select(dim='B', index=0)  # 取出第0張影像
x_0 = x.select(dim='B', index=1)
print('x_sum.size(): ', x_sum.size())
print('x.size(): ', x.size())
print('x_0.size(): ', x_0.size())

x_sum.size():  torch.Size([2, 1, 3, 3])
x.size():  torch.Size([2, 3, 3, 3])
x_0.size():  torch.Size([3, 3, 3])


In [66]:
x = torch.tensor([0, 1, 2 ,3 ,0])
y = torch.nonzero(x)
print('x: ', x)
print('y:\n', y, y.size())

x:  tensor([0, 1, 2, 3, 0])
y:
 tensor([[1],
        [2],
        [3]]) torch.Size([3, 1])


## Reshape

In [52]:
tensor1 = torch.empty([2, 3])
tensor2 = tensor1.view(-1)
print('tensor1:\n', tensor1.shape)
print('tensor2.shape: ', tensor2.shape)

tensor1:
 torch.Size([2, 3])
tensor2.shape:  torch.Size([6])


## 降維增維

In [53]:
tensor = torch.empty(5, 1)
print(tensor.shape)
tensor1 = torch.squeeze(tensor, axis=1)
print(tensor1.shape)
tensor2 = torch.squeeze(tensor, axis=0)
print(tensor2.shape)

torch.Size([5, 1])
torch.Size([5])
torch.Size([5, 1])


In [43]:
tensor = torch.empty(5)
print(tensor.shape)
t1 = torch.unsqueeze(tensor, axis=1)
print('t1.shape: ', t1.shape)
t2 = torch.unsqueeze(tensor, axis=0)
print('t2.shape: ', t2.shape)
t3 = tensor[:, None]
print('t3.shape: ', t3.shape)
t4 = tensor[None, :]
print('t4.shape: ', t4.shape)

torch.Size([5])
t1.shape:  torch.Size([5, 1])
t2.shape:  torch.Size([1, 5])
t3.shape:  torch.Size([5, 1])
t4.shape:  torch.Size([1, 5])


## torch.dist()

In [27]:
t1 = torch.tensor([1, 5, 3.])
t2 = torch.tensor([1, 2., 3])
print(f'{t1}和{t2}的距離: ', torch.dist(t1, t2))

tensor([1., 5., 3.])和tensor([1., 2., 3.])的距離:  tensor(3.)


## torch.cat()

In [33]:
t1 = torch.ones((1, 5, 5))
t2 = torch.ones((1, 5, 5))
t3 = torch.ones((1, 5, 5))
print('t1 ~ t3.shape: ', t1.shape)
x = torch.cat((t1, t2, t3), dim=0)
print('x.shape: ', x.shape)
x1 = torch.cat((t1, t2, t3), dim=1)
print('x1.shape: ', x1.shape)
x2 = torch.cat((t1, t2, t3), dim=2)
print('x2.shape: ', x2.shape)

t1 ~ t3.shape:  torch.Size([1, 5, 5])
x.shape:  torch.Size([3, 5, 5])
x1.shape:  torch.Size([1, 15, 5])
x2.shape:  torch.Size([1, 5, 15])


## torch.stack()

In [36]:
t1 = torch.ones((1, 5, 5))
t2 = torch.ones((1, 5, 5))
t3 = torch.ones((1, 5, 5))
print('t1 ~ t3.shape: ', t1.shape)
x = torch.stack((t1, t2, t3), dim=0)  #會新增一個維度
print('x.shape: ', x.shape)
x1 = torch.stack((t1, t2, t3), dim=1)
print('x1.shape: ', x1.shape)
x2 = torch.stack((t1, t2, t3), dim=2)
print('x2.shape: ', x2.shape)

t1 ~ t3.shape:  torch.Size([1, 5, 5])
x.shape:  torch.Size([3, 1, 5, 5])
x1.shape:  torch.Size([1, 3, 5, 5])
x2.shape:  torch.Size([1, 5, 3, 5])


## Torch & Numpy & list

In [16]:
np_data = np.arange(6).reshape((2, 3))
torch_data = torch.from_numpy(np_data)  
tensor2array = torch_data.numpy()
array2list =  np_data.tolist()
tensor = torch.tensor([1, 2, 3])
print('tensor.tolist(): ', tensor.tolist(), type(tensor.tolist()))
# 轉Numpy時必須確保是 在cpu上運算 且 requires_grad=False
# .detach(): requires_grad變為False
tensor_gpu_requires_grad2array= torch_data.cpu().detach().numpy()  
print('torch_data:\n', torch_data, type(torch_data))
print('\ntensor2array:\n', tensor2array, type(tensor2array))
print('array2list:\n', array2list, type(array2list))
print('tensor_gpu_requires_grad2array:\n', tensor_gpu_requires_grad2array, type(tensor_gpu_requires_grad2array))

tensor.tolist():  [1, 2, 3] <class 'list'>
torch_data:
 tensor([[0, 1, 2],
        [3, 4, 5]], dtype=torch.int32) <class 'torch.Tensor'>

tensor2array:
 [[0 1 2]
 [3 4 5]] <class 'numpy.ndarray'>
array2list:
 [[0, 1, 2], [3, 4, 5]] <class 'list'>
tensor_gpu_requires_grad2array:
 [[0 1 2]
 [3 4 5]] <class 'numpy.ndarray'>


## 簡單運算(和Numpy類似)

In [17]:
# abs 绝对值计算
data = [-1, -2, 1, 2]
tensor_int64 = torch.tensor(data)
print('tensor_int64:\n', tensor_int64, tensor_int64.dtype)
tensor_float32 = tensor_int64.type(torch.FloatTensor)  # .type()做 dtype的變換
print('tensor_float32:\n', tensor_float32, tensor_float32.dtype)
# tensor = torch.FloatTensor(data)  # 直接創建
print(
    '\nabs',
    '\nnumpy: ', np.abs(data),          # [1 2 1 2]
    '\ntorch: ', torch.abs(tensor_float32)      # [1 2 1 2]
)

# sin   三角函数 sin
print(
    '\nsin',
    '\nnumpy: ', np.sin(data),      # [-0.84147098 -0.90929743  0.84147098  0.90929743]
    '\ntorch: ', torch.sin(tensor_float32)  # [-0.8415 -0.9093  0.8415  0.9093]
)

# mean  均值
print(
    '\nmean',
    '\nnumpy: ', np.mean(data),         # 0.0
    '\ntorch: ', torch.mean(tensor_float32)     # 0.0
)

tensor_int64:
 tensor([-1, -2,  1,  2]) torch.int64
tensor_float32:
 tensor([-1., -2.,  1.,  2.]) torch.float32

abs 
numpy:  [1 2 1 2] 
torch:  tensor([1., 2., 1., 2.])

sin 
numpy:  [-0.84147098 -0.90929743  0.84147098  0.90929743] 
torch:  tensor([-0.8415, -0.9093,  0.8415,  0.9093])

mean 
numpy:  0.0 
torch:  tensor(0.)


## 常用屬性:
### tensor.float()  轉float32
### tensor.long()  轉int64

In [45]:
t1 = torch.tensor([1, 2, 3])
print('t1.dtype: ', t1.dtype)
t2 = t1.float()  
print('t2.dtype: ', t2.dtype)
t3 = t1.long()
print('t3.dtype: ', t3.dtype)

t1.dtype:  torch.int64
t2.dtype:  torch.float32
t3.dtype:  torch.int64


### tensor.clone()

In [44]:
t1 = torch.tensor([1, 2, 3])
t2 = t1.clone()
print(id(t1))
print(id(t2))

2220084371344
2220464854736


### tensor.clamp()

In [11]:
x = torch.rand(4)
print('x: ', x)
y = x.clamp(min=-0.5, max=0.5)
print('y: ', y)

x:  tensor([0.1033, 0.6847, 0.3334, 0.4812])
y:  tensor([0.1033, 0.5000, 0.3334, 0.4812])


### tensor.item()

In [13]:
x = torch.tensor([1])
print('x: ', x)
print('x.item(): ', x.item())  # 速度慢少用

x:  tensor([1])
x.item():  1


### tensor.detach()

In [25]:
t1 = torch.tensor([1., 2, 3], requires_grad=True)
print('t1: ', t1)
print('t1.detach(): ', t1.detach())  # set requires_grad=False

t1:  tensor([1., 2., 3.], requires_grad=True)
t1.detach():  tensor([1., 2., 3.])


## tensor.squeeze()

In [40]:
t = torch.empty((3, 1))
print('t.shape: ', t.shape)
print('t.squeeze(0).shape: ', t.squeeze(0).shape)
print('t.squeeze(1).shape: ', t.squeeze(1).shape)

t.shape:  torch.Size([3, 1])
t.squeeze(0).shape:  torch.Size([3, 1])
t.squeeze(1).shape:  torch.Size([3])


## tensor.unsqueeze()

In [41]:
t = torch.empty((3, 1))
print('t.shape: ', t.shape)
print('t.unsqueeze(0).shape: ', t.unsqueeze(0).shape)
print('t.unsqueeze(1).shape: ', t.unsqueeze(1).shape)

t.shape:  torch.Size([3, 1])
t.unsqueeze(0).shape:  torch.Size([1, 3, 1])
t.unsqueeze(1).shape:  torch.Size([3, 1, 1])


## tensor.sum()

In [43]:
x = torch.rand((3, 1, 5, 3))
y = x.sum(dim=0)
z = x.sum(dim=0, keepdim=True)
print('x.size(): ', x.size())
print('y.size(): ', y.size())
print('z.size(): ', z.size())

x.size():  torch.Size([3, 1, 5, 3])
y.size():  torch.Size([1, 5, 3])
z.size():  torch.Size([1, 1, 5, 3])


## tensor.select():

In [48]:
x = torch.rand((3, 1, 5, 5))
x1 = x.select(dim=0, index=0)  # 選出第一章影像
print('x.size(): ', x.size())
print('x1.size(): ', x1.size())  # 選出第一章影像

x.size():  torch.Size([3, 1, 5, 5])
x1.size():  torch.Size([1, 5, 5])


In [1]:
a = [1, 2, 3]
b = [1, 2, 3]
a == b

True